In [1]:
#salvar un modelo. se puede dejar donde estaba al entrenar y seguir donde estaba y compartirlo

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [3]:
import os
import tensorflow as tf
from tensorflow import keras

In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]
train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0 # normalizan solo las 1000 primeras

In [5]:
def create_model():
    model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')])
    model.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model

In [6]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [7]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1) # primero se crea para luego utilizarlo en el fit
model = create_model()


In [8]:
model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback]) # cada vez que hace un epoch salva los pesos

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 992/1000 [============================>.] - ETA: 0s - loss: 1.1519 - accuracy: 0.6653
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 1ms/sample - loss: 1.1520 - accuracy: 0.6650 - val_loss: 0.7075 - val_accuracy: 0.7750
Epoch 2/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.4348 - accuracy: 0.8761
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 624us/sample - loss: 0.4210 - accuracy: 0.8820 - val_loss: 0.5288 - val_accuracy: 0.8390
Epoch 3/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.2955 - accuracy: 0.9095
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 565us/sample - loss: 0.2893 - accuracy: 0.9140 - val_loss: 0.4642 - val_accuracy: 0.8550
Epoch 4/10
 864/1000 [========================>.....] - ETA: 0s - loss: 0.2188 - accuracy: 0.9468
Epoch 0

In [10]:
model = create_model()
loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 176us/sample - loss: 2.3342 - accuracy: 0.1170
Untrained model, accuracy: 11.70%


In [11]:
# cargamos  los pesos de un modelo entrenado al modelo nuevo
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)

1000/1000 [==============================] - 0s 96us/sample - loss: 0.3872 - accuracy: 0.8800


In [12]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt" # salva el modelo por epocas en distinto archivos
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)
model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [14]:
model = create_model()
latest = tf.train.latest_checkpoint(checkpoint_dir) # carga el ultimo archivo creado con los pesos. escoge el ultimo
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 145us/sample - loss: 0.4972 - accuracy: 0.8730
Restored model, accuracy: 87.30%


In [15]:
# salvando los pesos manualmente
model.save_weights('./checkpoints/my_checkpoint') # si no esta creada la carpeta, el la crea
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 150us/sample - loss: 0.4972 - accuracy: 0.8730
Restored model, accuracy: 87.30%


In [16]:
# se puede salvar el modelo completo con pesos, variables y la configuracion en un archivo HDF5 .h5 y recuperarlo para 
# tensorflow lite por ejemplo
model = create_model()
model.fit(train_images, train_labels, epochs=5)
model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 340us/sample - loss: 1.1349 - accuracy: 0.6830
Epoch 2/5
1000/1000 [==============================] - 0s 220us/sample - loss: 0.4203 - accuracy: 0.8860
Epoch 3/5
1000/1000 [==============================] - 0s 222us/sample - loss: 0.2991 - accuracy: 0.9210
Epoch 4/5
1000/1000 [==============================] - 0s 220us/sample - loss: 0.2129 - accuracy: 0.9490
Epoch 5/5
1000/1000 [==============================] - 0s 221us/sample - loss: 0.1519 - accuracy: 0.9620


In [18]:
#para cargarlo de nuevo
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [19]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 158us/sample - loss: 0.4223 - accuracy: 0.8620
Restored model, accuracy: 86.20%
